In [1]:
import pandas as pd
import os
import sys
from tqdm import tqdm
from IPython.display import clear_output


In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("yash07yadav/project-data")

print("Path to dataset files:", path)

Path to dataset files: /home/mcqueen/.cache/kagglehub/datasets/yash07yadav/project-data/versions/1


In [3]:
path = "/home/mcqueen/.cache/kagglehub/datasets/yash07yadav/project-data/versions/1"

In [4]:
new_path = os.path.join(path, "Complete Dataset")

In [5]:
train_data_path = os.path.join(path, "Complete Dataset", "train")
val_data_path = os.path.join(path, "Complete Dataset", "val")


hockeyFight_train_data_path = os.path.join(train_data_path, "HockeyFight")
movieFights_train_data_path = os.path.join(train_data_path, "MovieFights")
fight_train_data_path = os.path.join(train_data_path, "Fight")
nonFight_train_data_path = os.path.join(train_data_path, "NonFight")


hockeyFight_val_data_path = os.path.join(val_data_path, "HockeyFight")
movieFights_val_data_path = os.path.join(val_data_path, "MovieFights")
fight_val_data_path = os.path.join(val_data_path, "Fight")
nonFight_val_data_path = os.path.join(val_data_path, "NonFight")




In [6]:
print(len(os.listdir(hockeyFight_train_data_path)))
print(len(os.listdir(movieFights_train_data_path)))
print(len(os.listdir(fight_train_data_path)))
print(len(os.listdir(nonFight_train_data_path)))

print("-"*30)

print(len(os.listdir(hockeyFight_val_data_path)))
print(len(os.listdir(movieFights_val_data_path)))
print(len(os.listdir(fight_val_data_path)))
print(len(os.listdir(nonFight_val_data_path)))

400
188
1000
1000
------------------------------
100
35
275
275


In [7]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [ ]:
from backend.services.video_data_extraction.video_preprocessor import VideoDataExtractor
processor = VideoDataExtractor()

Using GPU: NVIDIA GeForce RTX 3050 Laptop GPU


In [9]:
from IPython.display import clear_output
from tqdm.auto import tqdm


In [10]:
def extract_data(directory_path: str, csv_path: str) -> None:
    """
    Extracts video data for every file in `directory_path`
    and appends the results to a CSV file.

    Parameters
    ----------
    directory_path : str
        Path to the folder containing video files.
    csv_path : str
        Path to the CSV file where extracted data will be written.
    """
    all_files = os.listdir(directory_path)

    for file_name in tqdm(all_files, desc="Processing videos"):
        num_interactions = processor.extract_video_data(
            os.path.join(directory_path, file_name),
            csv_path
        )
        # If you want to use num_interactions for logging or further processing:
        # print(f"{file_name}: {num_interactions}")


In [11]:
extract_data(hockeyFight_train_data_path, os.path.join("..","data" , "hockeyfight_train.csv"))


Processing videos:   0%|          | 0/400 [00:00<?, ?it/s]

Processing video: 360x288 at 25.0 fps
Using frame_skip: 1
Saved 241 interactions to ../data/hockeyfight_train.csv
Processing video: 360x288 at 25.0 fps
Using frame_skip: 1
Appended 109 interactions to ../data/hockeyfight_train.csv
Processing video: 360x288 at 25.0 fps
Using frame_skip: 1
Appended 42 interactions to ../data/hockeyfight_train.csv
Processing video: 360x288 at 25.0 fps
Using frame_skip: 1
Appended 7 interactions to ../data/hockeyfight_train.csv
Processing video: 360x288 at 25.0 fps
Using frame_skip: 1
Appended 13 interactions to ../data/hockeyfight_train.csv
Processing video: 360x288 at 25.0 fps
Using frame_skip: 1
Appended 7 interactions to ../data/hockeyfight_train.csv
Processing video: 360x288 at 25.0 fps
Using frame_skip: 1
Appended 3 interactions to ../data/hockeyfight_train.csv
Processing video: 360x288 at 25.0 fps
Using frame_skip: 1
Appended 146 interactions to ../data/hockeyfight_train.csv
Processing video: 360x288 at 25.0 fps
Using frame_skip: 1
Appended 381 inte

In [ ]:
extract_data(movieFights_train_data_path, os.path.join(DATA_DIR , "moviefight_train.csv"))

  0%|          | 0/188 [00:00<?, ?it/s]

Processing video: 1920x800 at 23.98 fps
Using frame_skip: 1
Saved 3 interactions to /home/ml/Documents/YoloV8-LSTM-video-Classification/data/moviefight_train.csv
Processing video: 1920x800 at 23.976 fps
Using frame_skip: 1
Appended 52 interactions to /home/ml/Documents/YoloV8-LSTM-video-Classification/data/moviefight_train.csv
Processing video: 720x576 at 25.0 fps
Using frame_skip: 1
Appended 42 interactions to /home/ml/Documents/YoloV8-LSTM-video-Classification/data/moviefight_train.csv
Processing video: 1920x816 at 23.98 fps
Using frame_skip: 1
Appended 311 interactions to /home/ml/Documents/YoloV8-LSTM-video-Classification/data/moviefight_train.csv
Processing video: 1920x800 at 23.976 fps
Using frame_skip: 1
Appended 47 interactions to /home/ml/Documents/YoloV8-LSTM-video-Classification/data/moviefight_train.csv
Processing video: 720x480 at 29.97002997002997 fps
Using frame_skip: 1
Appended 32 interactions to /home/ml/Documents/YoloV8-LSTM-video-Classification/data/moviefight_train.

In [ ]:
extract_data(fight_train_data_path, os.path.join(DATA_DIR , "fight_train.csv"))


In [ ]:
extract_data(nonFight_train_data_path, os.path.join(DATA_DIR , "non-fight_train.csv"))


validation dataset

In [ ]:
extract_data(hockeyFight_val_data_path, os.path.join(DATA_DIR , "hockeyfight_val.csv"))


In [ ]:
extract_data(movieFights_val_data_path, os.path.join(DATA_DIR , "moviefight_val.csv"))


In [ ]:
extract_data(fight_val_data_path, os.path.join(DATA_DIR , "fight_val.csv"))

In [ ]:
extract_data(nonFight_val_data_path, os.path.join(DATA_DIR , "non-fight_val.csv"))